# Sentiment Analysis with Deep Learning

# Phase 2- Modelling

This notebook consists the functions and code for modelling.  

### CHRISP-DM phases

Modelling and Evaluation phases for CRISP-DM can be found in this noteboook.

#### 4.Modeling
Modeling techniques are selected and applied. 

#### 5.Evaluation
Once one or more models have been built that appear to have high quality based on whichever loss functions have been selected, these need to be tested to ensure they generalize against unseen data and that all key business issues have been sufficiently considered.  The end result is the selection of the champion model(s).

### Table of Contents

- 1.Import Libraries
- 2.Define Functions 
- 3.Modeling With Neural Networks  
- 4.Tuning the Best Model

## 1. Import Libraries

In [2]:
import pandas as pd
import gc
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sns
import nltk
from nltk import word_tokenize, FreqDist
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
np.random.seed(0)
import pickle

In [3]:
from keras.models import Model, Sequential, Input
from keras.layers import Dense, Embedding, Input, Conv1D, GlobalMaxPool1D, GlobalAveragePooling1D, Dropout, concatenate, Layer, InputSpec, CuDNNLSTM, SpatialDropout1D, Activation, LSTM
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import activations, initializers, regularizers, constraints, optimizers, layers
from keras.utils.conv_utils import conv_output_length
from keras.regularizers import l2
from keras.constraints import maxnorm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, SpatialDropout1D, Activation
from keras.layers import Conv1D, Bidirectional, GlobalMaxPool1D, MaxPooling1D, BatchNormalization
from keras.optimizers import Adam, SGD
import pickle

Using TensorFlow backend.


## 2. Functions 

In [4]:
# number of unique words we want to use (or: number of rows in incoming embedding vector)
max_features = 8192
# max number of words in a comment to use (or: number of columns in incoming embedding vector)
max_len = 128
# dimension of the embedding variable (or: number of rows in output of embedding vector)
embedding_dims = 64

In [5]:
def load_smalldata ( ):
    # Loading partial train test files , tokenized, sequenzed and padded
    pickle_in = open("data/vectors_small/X_train2_file.pickle","rb")
    X_train2 = pickle.load(pickle_in)

    pickle_in = open("data/vectors_small/X_test2_file.pickle","rb")
    X_test2 = pickle.load(pickle_in)

    pickle_in = open("data/vectors_small/y_train2_file.pickle","rb")
    y_train2 = pickle.load(pickle_in)

    pickle_in = open("data/vectors_small/y_test2_file.pickle","rb")
    y_test2 = pickle.load(pickle_in)
    
    return X_train2, X_test2, y_train2, y_test2


def load_tokenizer():
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer=pickle.load(handle)
        return tokenizer
    
def create_model ( hidden_layers, 
                  loss='binary_crossentropy',
                  optimizer=Adam(0.01),
                  metrics=['accuracy'],
                  embedding_matrix=None,
                  max_len=max_len,
                  embedding_dims=embedding_dims,
                  max_features=max_features,
                  glove=False,
                 ):
 
    # check if embedding matrix has assigned which means the model uses glove embeddings 
    if glove==False:
        emb_layer=Embedding(input_dim=max_features, input_length=max_len,
                        output_dim=embedding_dims)
    else:
        
        emb_layer=Embedding(input_dim =embedding_matrix.shape[0], input_length=max_len,
                          output_dim=embedding_matrix.shape[1], 
                          weights=[embedding_matrix], trainable=False)
    
    # instantiate Sequential model
    model = Sequential()
 
    # add embedding layer with defined parameters
    model.add(emb_layer)
   
    # add hidden layers available in hidden_layers list
    for layer in hidden_layers:
        model.add(layer)
    
    # add pooling layer 
    model.add(GlobalMaxPool1D())

    # set the dropout layer to drop out 50% of the nodes
    model.add(Dropout(0.5))

    # add dense layer to produce an output dimension of 50 and using relu activation
    model.add(Dense(50, activation='relu'))

    # finally add a dense layer
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=metrics)
    
    model.summary() 
    return model


def run_model(model, model_name, results, epochs, batch_size=32):
    hist = model.fit(X_train2, y_train2, 
                     batch_size=batch_size, 
                     epochs=epochs, 
                     validation_split=0.1)

    test_loss, test_auc = model.evaluate(X_test2, y_test2, batch_size=32)
    print("-------------------------------------------")
    print("")
    print("")
    print(model_name + ' Test Loss:    ', test_loss)
    print(model_name + ' Test Accuracy:', test_auc)
    print("")
    print("")
    print("")
    
    
    #Pass the results as key value pairs to append() function 
    row=[]
    row =[model_name , hist.history['accuracy'][-1],
          hist.history['val_accuracy'][-1],test_auc, test_loss] 

    save_model(model, model_name)
    return row



def save_model (model, model_name):
    path="models/"+model_name + ".h5"
    model.save(path)

def save_results (results, row):
    #Pass the results as key value pairs to append() function 
    results = results.append({'model' : row[0] , 
                        'train_acc' : row[1], 
                        'val_acc':row[2],
                        'test_acc': row[3]
                              
                                    } , ignore_index=True)
    return results


### Loading Glove Dictionary
def load_glove (path):
    # load the glove840B embedding
    embeddings_index = dict()
    f = open(path)

    for line in f:
        # Note: use split(' ') instead of split() if you get an error
        values = line.split(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Loaded %s word vectors.' % len(embeddings_index))

    # create a weight matrix
    embedding_matrix = np.zeros((len(tokenizer.word_index)+1, 300))

    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix


## 3. Modeling With Neural Networks                       

I will first try 4 different models to see which one gives the best result.

#### Set parameters for modeling

#### Create a dataframe to store the accuarecy results

In [13]:
results=pd.DataFrame(columns=["model", "train_acc","val_acc" ,"test_acc"])

In [7]:
#load small dataset to train and test the models
X_train2, X_test2, y_train2, y_test2=load_smalldata()
#load tokenizer
tokenizer=load_tokenizer()

#### Define parameters for each model such as hidden layers and glove

In [8]:
dnn_params={'hidden_layers':[],
            'glove' : False,
            'embedding_matrix':None
           }


cnn_params = {
        'hidden_layers':[SpatialDropout1D(0.5),
                   Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'),
                   BatchNormalization()],
        'glove' : False,
        'embedding_matrix':None}


rnn_params ={ 
     'hidden_layers':[SpatialDropout1D(0.5),
                  Bidirectional(LSTM(25, 
                  return_sequences=True))],
     'glove' : False,
     'embedding_matrix':None}

#load glove embedding vectors from txt file
embedding_matrix=load_glove ("glove.6B.300d.txt")

cnn_glove_params ={
    'hidden_layers':[SpatialDropout1D(0.5),
                   Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'),
                   BatchNormalization()],
    'glove' : True,
    
    'embedding_matrix':embedding_matrix
    }

rnn_glove_params ={
    'hidden_layers':[SpatialDropout1D(0.5),
                  Bidirectional(LSTM(25, 
                  return_sequences=True))],
    'glove' : True,
    'embedding_matrix':embedding_matrix
    }


Loaded 400000 word vectors.


In [14]:
# Create lists for model names and hiden layer
names=["dnn","cnn", "rnn", "cnn_glove", "rnn_glove"]
params=[dnn_params, cnn_params, rnn_params, 
        cnn_glove_params, rnn_glove_params]
epochs=3

#### For loop for modelling

Run a for loop to create the model, compile, fit and save results and the model itsef

In [15]:
#set epochs to 3
#Run a for loop to create the model, compile, fit and save results 
#and save the model to models folder
for name, param in zip(names, params):
    print("Model Name :", name)
    print("======================")
    model=create_model(hidden_layers=param['hidden_layers'], 
                   glove=param['glove'],
                   embedding_matrix=param['embedding_matrix']
                    )
    row=run_model(model, name, results, epochs)
    results=save_results(results, row)


Model Name : dnn
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 128, 64)           524288    
_________________________________________________________________
global_max_pooling1d_9 (Glob (None, 64)                0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 50)                3250      
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 51        
Total params: 527,589
Trainable params: 527,589
Non-trainable params: 0
_________________________________________________________________
3


/Users/kaan/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 135000 samples, validate on 15000 samples
Epoch 1/3
135000/135000 [==============================] - 56s 414us/step - loss: 0.3914 - accuracy: 0.8251 - val_loss: 0.3244 - val_accuracy: 0.8681
Epoch 2/3
135000/135000 [==============================] - 59s 440us/step - loss: 0.3341 - accuracy: 0.8565 - val_loss: 0.2909 - val_accuracy: 0.8749
Epoch 3/3
30000/30000 [==============================] - 2s 60us/step
-------------------------------------------


dnn Test Loss:     0.30266345343589784
dnn Test Accuracy: 0.8708333373069763



Model Name : cnn
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 128, 64)           524288    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 128, 64)           0         
_________________________________________________________________
conv1d_1 (Conv1D)

/Users/kaan/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 135000 samples, validate on 15000 samples
Epoch 1/3
135000/135000 [==============================] - 206s 2ms/step - loss: 0.3754 - accuracy: 0.8282 - val_loss: 0.2835 - val_accuracy: 0.8812
Epoch 2/3
135000/135000 [==============================] - 182s 1ms/step - loss: 0.2934 - accuracy: 0.8795 - val_loss: 0.2708 - val_accuracy: 0.8869
Epoch 3/3
30000/30000 [==============================] - 10s 320us/step
-------------------------------------------


cnn Test Loss:     0.2730163905858993
cnn Test Accuracy: 0.8843333125114441



Model Name : rnn
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 128, 64)           524288    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 128, 64)           0         
_________________________________________________________________
bidirectional_1 (B

/Users/kaan/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 135000 samples, validate on 15000 samples
Epoch 1/3
135000/135000 [==============================] - 636s 5ms/step - loss: 0.4429 - accuracy: 0.7929 - val_loss: 0.3318 - val_accuracy: 0.8555
Epoch 2/3
135000/135000 [==============================] - 568s 4ms/step - loss: 0.3793 - accuracy: 0.8332 - val_loss: 0.3119 - val_accuracy: 0.8665
Epoch 3/3
30000/30000 [==============================] - 28s 930us/step
-------------------------------------------


rnn Test Loss:     0.31319638199806216
rnn Test Accuracy: 0.8661333322525024



Model Name : cnn_glove
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 128, 300)          263894100 
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 128, 300)          0         
_________________________________________________________________
conv1d_2 (C

## Checking results and picking the model for tuning


In [16]:
results

,model,train_acc,val_acc,test_acc
0,dnn,0.864948,0.875733,0.870833
1,cnn,0.889348,0.887933,0.884333
2,rnn,0.844452,0.869733,0.866133
3,cnn_glove,0.827726,0.867600,0.866600
4,rnn_glove,0.816000,0.852733,0.852000


Based on this results CNN is the best performing model so far. From this forward, I will tune only this model and try to impove its performance. 

## 4. Tuning the Best Model

## Iteration 1:  CNN with SGD optimizer

I will use a different optimizer for the same model structure. I will use SGD.  

In [18]:
sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.5, nesterov=True)

In [19]:
# iteration 1 = with sgd optimizer
model=create_model(hidden_layers=[SpatialDropout1D(0.5),
                        Conv1D(filters=100,kernel_size=4, padding='same', activation='relu'),
                        BatchNormalization()],
                   
                      glove = False,
                      embedding_matrix=None,
                      optimizer = sgd)
row=run_model(model, "cnn_glove_sgd", results, epochs)

results=save_results(results, row)
 

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 128, 64)           524288    
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 128, 64)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 128, 100)          25700     
_________________________________________________________________
batch_normalization_4 (Batch (None, 128, 100)          400       
_________________________________________________________________
global_max_pooling1d_14 (Glo (None, 100)               0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 50)              

/Users/kaan/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 135000 samples, validate on 15000 samples
Epoch 1/3
135000/135000 [==============================] - 156s 1ms/step - loss: 0.4958 - accuracy: 0.7368 - val_loss: 0.3542 - val_accuracy: 0.8494
Epoch 2/3
135000/135000 [==============================] - 148s 1ms/step - loss: 0.3541 - accuracy: 0.8474 - val_loss: 0.3149 - val_accuracy: 0.8705
Epoch 3/3
30000/30000 [==============================] - 10s 317us/step
-------------------------------------------


cnn_glove_sgd Test Loss:     0.33474224851926165
cnn_glove_sgd Test Accuracy: 0.8713666796684265





SGD optimizer did not perform better than "adam". So lets stick with "adam" optimizer.  

## Iteration 2:  CNN with 2 convolution layers


In [25]:
# iteration 1 = with sgd optimizer
model=create_model(hidden_layers=[SpatialDropout1D(0.5),
                        Conv1D(filters=100,kernel_size=4, padding='same', activation='relu'),
                        BatchNormalization(),
                        Conv1D(filters=100,kernel_size=4, padding='same', activation='relu'),
                        BatchNormalization()],
                        glove =False,
                        embedding_matrix=None,
                      )
epochs=3
row=run_model(model, "cnn_2cnv", results, epochs)

results=save_results(results, row)
 

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 128, 64)           524288    
_________________________________________________________________
spatial_dropout1d_13 (Spatia (None, 128, 64)           0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 128, 100)          25700     
_________________________________________________________________
batch_normalization_14 (Batc (None, 128, 100)          400       
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 128, 100)          40100     
_________________________________________________________________
batch_normalization_15 (Batc (None, 128, 100)          400       
_________________________________________________________________
global_max_pooling1d_19 (Glo (None, 100)             

/Users/kaan/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 135000 samples, validate on 15000 samples
Epoch 1/3
135000/135000 [==============================] - 342s 3ms/step - loss: 0.6993 - accuracy: 0.5014 - val_loss: 0.6931 - val_accuracy: 0.5012
Epoch 2/3
135000/135000 [==============================] - 356s 3ms/step - loss: 0.6934 - accuracy: 0.5003 - val_loss: 0.6932 - val_accuracy: 0.5012
Epoch 3/3
30000/30000 [==============================] - 20s 669us/step
-------------------------------------------


cnn_2cnv Test Loss:     0.6931803480148315
cnn_2cnv Test Accuracy: 0.4967666566371918





With the second convolution layer cnn model did not learn at all. 

Adding a 2nd convolution layer did not improved the model. 

## Iteration 3:  CNN_Glove  with more epochs

In [31]:

model=create_model(hidden_layers=[SpatialDropout1D(0.5),
                        Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'),
                        BatchNormalization()],
                      glove = False,
                      embedding_matrix=None)
epochs=8
row=run_model(model, "cnn_glove_8epochs", results, epochs)
results=save_results(results, row)
 

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (None, 128, 64)           524288    
_________________________________________________________________
spatial_dropout1d_22 (Spatia (None, 128, 64)           0         
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 128, 100)          25700     
_________________________________________________________________
batch_normalization_24 (Batc (None, 128, 100)          400       
_________________________________________________________________
global_max_pooling1d_25 (Glo (None, 100)               0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 50)              

/Users/kaan/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 135000 samples, validate on 15000 samples
Epoch 1/8
135000/135000 [==============================] - 173s 1ms/step - loss: 0.4782 - accuracy: 0.7417 - val_loss: 0.2934 - val_accuracy: 0.8789
Epoch 2/8
135000/135000 [==============================] - 164s 1ms/step - loss: 0.3023 - accuracy: 0.8755 - val_loss: 0.2747 - val_accuracy: 0.8859
Epoch 3/8
135000/135000 [==============================] - 164s 1ms/step - loss: 0.2771 - accuracy: 0.8877 - val_loss: 0.2718 - val_accuracy: 0.8866
Epoch 4/8
135000/135000 [==============================] - 173s 1ms/step - loss: 0.2625 - accuracy: 0.8954 - val_loss: 0.2709 - val_accuracy: 0.8906
Epoch 5/8
135000/135000 [==============================] - 173s 1ms/step - loss: 0.2523 - accuracy: 0.8993 - val_loss: 0.2618 - val_accuracy: 0.8907
Epoch 6/8
135000/135000 [==============================] - 174s 1ms/step - loss: 0.2405 - accuracy: 0.9054 - val_loss: 0.2717 - val_accuracy: 0.8871
Epoch 7/8
135000/135000 [==============================

In [32]:
results

,model,train_acc,val_acc,test_acc
0,dnn,0.864948,0.875733,0.870833
1,cnn,0.889348,0.887933,0.884333
2,rnn,0.844452,0.869733,0.866133
3,cnn_glove,0.827726,0.867600,0.866600
4,rnn_glove,0.816000,0.852733,0.852000
5,cnn_glove_sgd,0.863733,0.874933,0.871367
6,cnn_2cnv,0.499422,0.501200,0.496767
7,cnn_glove_8epochs,0.913407,0.888733,0.886167
